In [1]:
import os
import cv2
import pandas as pd
from glob import glob

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from time import time

# 基本設定

In [2]:
data_dir_path = './'

# 建立 target labels 清單

In [3]:
target_label_file_name = 'mapping.txt'

target_label_file_path = '/'.join((data_dir_path, target_label_file_name))

with open(target_label_file_path) as f:
    all_lines = [line.split(', ') for line in f.read().splitlines()]

target_labels = dict()
for line in all_lines:
    target_class, target_label = line
    target_labels[target_class] = target_label

In [4]:
target_labels

{'CALsuburb': '9',
 'PARoffice': '7',
 'bedroom': '12',
 'coast': '10',
 'forest': '4',
 'highway': '14',
 'industrial': '2',
 'insidecity': '3',
 'kitchen': '0',
 'livingroom': '5',
 'mountain': '8',
 'opencountry': '6',
 'store': '11',
 'street': '1',
 'tallbuilding': '13'}

# 建立資料清單

In [10]:
# 指定存放 train 資料集的資料夾
train_dir = '/'.join((data_dir_path, 'train'))
img_path_list = []
img_class_list = []
for key in target_labels.keys():
    for file_path in glob('{}/{}/*.jpg'.format(train_dir, key)):
        img_class_list.append(target_labels[key])
        img_path_list.append(file_path)

data_list = pd.DataFrame({'class': img_class_list, 'path': img_path_list})

In [11]:
data_list.head(10)

class                               path
0     9  .//train/CALsuburb\image_0001.jpg
1     9  .//train/CALsuburb\image_0002.jpg
2     9  .//train/CALsuburb\image_0003.jpg
3     9  .//train/CALsuburb\image_0004.jpg
4     9  .//train/CALsuburb\image_0005.jpg
5     9  .//train/CALsuburb\image_0006.jpg
6     9  .//train/CALsuburb\image_0007.jpg
7     9  .//train/CALsuburb\image_0008.jpg
8     9  .//train/CALsuburb\image_0009.jpg
9     9  .//train/CALsuburb\image_0010.jpg

## Shuffle and split the list into training set and validation set

In [12]:
# 將資料清單隨機打亂
rand_seed = int(time())
data_list = shuffle(data_list, random_state=rand_seed)
data_list.head(10)

class                                  path
1967     8      .//train/mountain\image_0230.jpg
2515    11         .//train/store\image_0194.jpg
2413    11         .//train/store\image_0092.jpg
2364    11         .//train/store\image_0043.jpg
656      4        .//train/forest\image_0025.jpg
2079     6   .//train/opencountry\image_0068.jpg
2314     6   .//train/opencountry\image_0303.jpg
2853    13  .//train/tallbuilding\image_0125.jpg
967     14       .//train/highway\image_0108.jpg
389     10         .//train/coast\image_0018.jpg

In [13]:
X = pd.DataFrame(data_list['path'], columns=['path'])
y = pd.DataFrame(data_list['class'], columns=['class'])
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25)

經過 train_test_split 切割出來的 train 及 test 資料，samples 跟 labels 清單的 index 順序是一樣的，可以安心拿來訓練 model。

In [14]:
print('Some training samples:\n----------------------')
print(X_train.head())

print('\nSome training labels:\n---------------------')
print(y_train.head())

print('\nSome validation samples:\n------------------------')
print(X_valid.head())

print('\nSome validation labels:\n-----------------------')
print(y_valid.head())

Some training samples:
----------------------
                                      path
30       .//train/CALsuburb\image_0031.jpg
640         .//train/forest\image_0009.jpg
1209    .//train/industrial\image_0190.jpg
209      .//train/PARoffice\image_0069.jpg
2905  .//train/tallbuilding\image_0177.jpg

Some training labels:
---------------------
     class
30       9
640      4
1209     2
209      7
2905    13

Some validation samples:
------------------------
                                      path
1328    .//train/insidecity\image_0098.jpg
2751  .//train/tallbuilding\image_0023.jpg
563          .//train/coast\image_0192.jpg
2494         .//train/store\image_0173.jpg
1842      .//train/mountain\image_0105.jpg

Some validation labels:
-----------------------
     class
1328     3
2751    13
563     10
2494    11
1842     8


把 DataFrame 存為 .csv 檔

In [15]:
# Save to csv
X_train.to_csv('X_train.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
X_valid.to_csv('X_valid.csv', index=False)
y_valid.to_csv('y_valid.csv', index=False)

建立用來儲存模型的資料夾

In [16]:
path = 'saved_models'
if not os.path.exists('saved_models'):
    os.mkdir('saved_models')